In [1]:
from sklearn.datasets import load_boston
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import roc_auc_score,  roc_curve
from matplotlib import pyplot
from sklearn.metrics import auc
import seaborn as sns
from sklearn import metrics
import datetime
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
import warnings

print("xgb.__version__ : ",xgb.__version__)
data_dir= '/home/lpatel/projects/AKI/data_592v'
#data_dir= '~/projects/AKI/test'
#data_dir='/home/lpatel/projects/AKI/data'
train_csv = os.path.join(data_dir,'train_csv.csv')
test_csv = os.path.join(data_dir,'test_csv.csv')
weight_csv = os.path.join(data_dir,'weight_csv.csv')

train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)
weight = pd.read_csv(weight_csv)
#column names are formted inconsitantly 
weight['col_fmt'] = weight.col.str.replace('-','.').str.replace(':','.')


cols = train.columns.tolist()
X_col = cols[1:-1]
y_col = cols[-1]

X_train,y_train = train[X_col],train[y_col]
X_test,  y_test = test[X_col] ,test[y_col]

print(set(X_col) -set(weight.col_fmt.tolist()) )
print(set(weight.col_fmt.tolist()) - set(X_col) )

weight1_lst =  weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight1.tolist()
weight2_lst =  weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight2.tolist()
weight3_lst =  weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight3.tolist()
weight4_lst =  weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight4.tolist()
weight5_lst =  weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight5.tolist()


xgb.__version__ :  1.0.0-SNAPSHOT
set()
set()


In [ ]:
# fit model no training data
params = {'booster': 'gbtree', 'max_depth': 10, 'min_child_weight': 10, 'eta': 0.3, 'objective': 'binary:logistic', 'n_jobs': 20, 'silent': True, 'eval_metric': 'logloss', 'subsample': 0.8, 'colsample_bytree': 0.5000000007978516, 'seed': 1001}
model = xgb.XGBClassifier(
    n_jobs = 20,
    params = params
)
model.fit(X_train, y_train)
print(model.get_xgb_params)
df= pd.DataFrame({'cols':X_train.columns,'feature_importances' :model.feature_importances_ }).sort_values(by='feature_importances',ascending=False)
t = datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
df.to_csv("/home/lpatel/aki/results/feature_importance_tesing.csv"+t+'_w1',index=False)
exit(0)

In [2]:
# def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
#                        model, param_grid, cv=10, scoring_fit = 'roc_auc',
#                        do_probabilities = True):
    
#     gs = GridSearchCV(
#         estimator=model,
#         param_grid=param_grid, 
#         cv=cv, 
#         n_jobs=4, 
#         scoring=scoring_fit,
#         verbose=2
#     )
#     fitted_model = gs.fit(X_train_data, y_train_data)
    
#     if do_probabilities:
#         pred = fitted_model.predict_proba(X_test_data)
#     else:
#         pred = fitted_model.predict(X_test_data)
    
#     return fitted_model, pred

# model = xgb.XGBClassifier(
#     objective='binary:logistic',
#     n_jobs = 6
# )
# param_grid = {
#     'max_depth': [3, 6, 9],
#     'n_estimators': [500, 1000, 1500],
#     'colsample_bytree': [0.05,0.5,0.75],
#     'subsample': [0.5, 0.75, 0.9],
#     'objective': ['binary:logistic'],

# }

# # ddddddddddddddddddd


# model, pred  = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
#                                  param_grid, cv=5)

# data = pd.DataFrame(model.cv_results_)
# # pd.options.display.max_columns = None
# # pd.options.display.max_rows = None
# print(data)
# t = datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
# data.to_csv("~/results_parm_cv.csv_weight1_lst" + t)
# print ("done")

In [3]:
AUC_LIST = []
LOG_LOSS_LIST = []
ITERbest_LIST = []
PARAM_LIST = []

dtrain = xgb.DMatrix(X_train, label = y_train)


def XGB_CV(max_depth,
          # n_estimators, 
           colsample_bytree, subsample, min_child_weight,eta):


    global AUC_LIST
    global LOG_LOSS_LIST
    global ITERbest_LIST
    global PARAM_LIST
    
    #print(n_estimators)

    paramt = {
              'booster' : 'gbtree',
              'max_depth' :  int(max_depth),
              'min_child_weight' : int(min_child_weight),
#               'n_estimators': int(n_estimators),
              'eta' : float(eta),
              'objective' : 'binary:logistic',
              'n_jobs' : 20,
              'silent' : True,
              'eval_metric': 'logloss',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'seed' : 1001
              }
    
    PARAM_LIST.append(paramt)

    folds = 5
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file )
    log_file.flush()

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    #num_boost_round = int(n_estimators),
                    stratified = True,
                    nfold = folds,
                    early_stopping_rounds = 100,
                    metrics = ['auc', 'logloss'],
                    show_stdv = True
               )



    auc_score = xgbc['test-auc-mean'].iloc[-1]
    logloss_score = xgbc['test-logloss-mean'].iloc[-1]
    iterbest = len(xgbc)
    AUC_LIST.append(auc_score)
    LOG_LOSS_LIST.append(logloss_score)
    ITERbest_LIST.append(iterbest)
    

    return (auc_score*2) - 1



XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (4, 10),
#                                      'n_estimators': (1, 10),
                                     'colsample_bytree': (0.5, 0.9),
                                     'subsample': (0.5, 0.8),
                                     'min_child_weight':(1,10),
                                     'eta':(0.05,0.3)
                                    })

In [4]:
t = datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
log_file = open('/home/lpatel/aki/results/test.log'+t, 'a')
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=10, n_iter=100)

|   iter    |  target   | colsam... |    eta    | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.4608   |  0.6747   |  0.08035  |  6.962    |  3.543    |  0.5251   |
|  2        |  0.5617   |  0.7961   |  0.2252   |  8.711    |  7.089    |  0.7615   |
|  3        |  0.5278   |  0.836    |  0.2338   |  6.587    |  6.334    |  0.7527   |
|  4        |  0.458    |  0.8588   |  0.211    |  5.031    |  2.583    |  0.7612   |
|  5        |  0.5187   |  0.886    |  0.267    |  6.684    |  1.401    |  0.5772   |
|  6        |  0.5085   |  0.6191   |  0.2461   |  5.187    |  7.596    |  0.653    |
|  7        |  0.5135   |  0.5993   |  0.1063   |  7.333    |  9.847    |  0.587    |
|  8        |  0.5262   |  0.5582   |  0.1383   |  9.903    |  2.868    |  0.6627   |
|  9        |  0.4596   |  0.6658   |  0.2797   |  4.182    |  2.648    |  0.5712   |
|  10       |  0.5142   |  0.7248   |  0.2289   |  6.9

|  58       |  0.5493   |  0.5057   |  0.2908   |  7.238    |  2.103    |  0.6868   |
|  59       |  0.5496   |  0.5347   |  0.2856   |  7.155    |  6.158    |  0.5336   |
|  60       |  0.5205   |  0.5302   |  0.2996   |  5.018    |  9.88     |  0.5351   |
|  61       |  0.5638   |  0.5571   |  0.2983   |  9.986    |  8.19     |  0.6342   |
|  62       |  0.5164   |  0.5096   |  0.2965   |  5.284    |  4.58     |  0.7925   |
|  63       |  0.5283   |  0.5      |  0.3      |  7.798    |  1.74     |  0.5      |
|  64       |  0.5801   |  0.5226   |  0.2996   |  9.006    |  9.996    |  0.7876   |
|  65       |  0.573    |  0.5702   |  0.2722   |  8.634    |  8.191    |  0.7941   |
|  66       |  0.5394   |  0.8887   |  0.2875   |  7.827    |  6.693    |  0.7839   |
|  67       |  0.5643   |  0.5411   |  0.2947   |  9.571    |  5.768    |  0.5105   |
|  68       |  0.5512   |  0.5      |  0.3      |  8.464    |  7.233    |  0.5      |
|  69       |  0.4753   |  0.5265   |  0.06307  |  6.4

In [ ]:
df = pd.DataFrame({"auc": AUC_LIST, "log": LOG_LOSS_LIST, "round": ITERbest_LIST, "param": PARAM_LIST })
df['param'] =  df['param'].astype(str)

t = datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
df.to_csv("/home/lpatel/aki/results/cv_result_baysian.csv"+t, sep="|")

In [6]:
print (len(ITERbest_LIST),len(PARAM_LIST),len(LOG_LOSS_LIST),len(AUC_LIST))

110 110 110 110


In [7]:
print(weight1_lst)

[0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.000985221674876847, 0.0032258064516129, 0.000985221674876847, 0.0032258064516129, 0.00322580645

In [8]:
print(weight2_lst)

[0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.000738916256157635, 0.00376344086021505, 0.000738916256157635, 0.00376344086021505, 0.003763440

In [9]:
print(weight3_lst)

[0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.000492610837438424, 0.0043010752688172, 0.000492610837438424, 0.0043010752688172, 0.00430107526

In [10]:
print(weight4_lst)

[0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.000246305418719212, 0.00483870967741936, 0.000246305418719212, 0.00483870967741936, 0.004838709

In [11]:
print(weight5_lst)

[2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 2.46305418719212e-05, 0.00532258064516129, 2.46305418719212e-05, 0.00532258064516129, 0.005322580

In [42]:
w5_string = weight.set_index(keys=['col_fmt']).reindex(X_train.columns.tolist()).weight5.tolist()
print(["{0:.7f}".format(i) for i in w5_string])

['0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0053226', '0.0000246', '0.0053226', '0.0053226', '0.0000246', '0.0000246', '0.0000246', '0.0053226', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0053226', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0000246', '0.0053226', '0.0000246', '0.0000246', '0.0053226', '0.0053226', '0.0053226', '0.0000246'

In [15]:
print([1/len(weight1_lst) for i in range(len(weight1_lst))])

[0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.0016891891891891893, 0.00168918

<bound method XGBModel.get_xgb_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=20,
              objective='binary:logistic',
              params={'booster': 'gbtree',
                      'colsample_bytree': 0.5000000007978516, 'eta': 0.3,
                      'eval_metric': 'logloss', 'max_depth': 10,
                      'min_child_weight': 10, 'n_jobs': 20,
                      'objective': 'binary:logistic', 'seed': 1001,
                      'silent': True, 'subsample': 0.8},
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='auto', validate_parameters=False,
              verbosity=1)>


In [5]:
df

,cols,feature_importances
509,CH.71010,0.027257
204,2160.0_change,0.022644
108,1719286.01,0.021794
568,CH.J7060,0.016275
321,373112.01_cum,0.016081
...,...,...
226,257,0.000000
225,256,0.000000
224,255,0.000000
222,252,0.000000
